In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)


Bad key "text.kerning_factor" on line 4 in
C:\Users\ranga\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data_train = pd.read_csv('train_LZdllcl.csv')
data_test = pd.read_csv('test_2umaH9m.csv')

In [3]:
data_train_copy = data_train.copy(deep = True)
data_test_copy = data_test.copy(deep = True)

In [4]:
def eda(dataset):
    data = dataset.copy(deep = True)
    data.previous_year_rating.fillna(3, inplace = True)   # add main
    data.education.fillna("Bachelor's" ,inplace = True)  ##add main
    
    data = data.drop(columns= ['employee_id','region'])   #add main

    data['awards_won?'] = data['awards_won?'].astype('object')
    data['KPIs_met >80%'] = data['KPIs_met >80%'].astype('object')
#     data['awards_won?'] = data['awards_won?'].astype('object')
    
    data = pd.get_dummies(data)

    return data

In [5]:
def under_sample(X_train, y_train, random_under_sample = True,random_replacement = False ,ver = 1):
    # X_us = pd.DataFrame(impute(X_train), columns = X_train.columns,index = X_train.index)  #smote does not accept null values, impute returns array
    X_us = X_train
    y_us = y_train.copy()

    from imblearn.under_sampling import RandomUnderSampler,NearMiss
    if random_under_sample:
        us = RandomUnderSampler(random_state=0, replacement = random_replacement)
        columns = X_us.columns

        X_train_us,y_train_us=us.fit_sample(X_us, y_us)
        X_train_us = pd.DataFrame(data=X_train_us,columns=columns ) # both index changes , i think we dont need to change index
    
    else:
        us = NearMiss(version = ver)
        columns = X_us.columns

        X_train_us,y_train_us=us.fit_sample(X_us, y_us)
        X_train_us = pd.DataFrame(data=X_train_us,columns=columns )
        
        
    return X_train_us, y_train_us

In [6]:
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier
from xgboost import XGBRFClassifier,XGBClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc,recall_score,classification_report

In [7]:
data_eda = eda(data_train)

In [8]:
X = data_eda.drop(columns=['is_promoted'])
y = data_eda.is_promoted

X_train, X_val, y_train, y_val = train_test_split(X,y, random_state = 6, train_size = .8)

X_train_us, y_train = under_sample(X_train,y_train)
X_train = np.round(X_train_us)

In [9]:
model =RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,   ## add main
                       criterion='gini', max_depth=50, max_features=20,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=61,
                       verbose=0, warm_start=False)

model.fit(X_train, y_train)

y_predict = model.predict(X_val)
print(confusion_matrix(y_val,y_predict)) 
print(accuracy_score(y_val,y_predict))
    
fpr, tpr, thresholds = roc_curve(y_val, y_predict, pos_label=None)
print('auc :',auc(fpr, tpr))
print('recall :', recall_score(y_val, y_predict))
print(classification_report(y_val, y_predict))

[[7002 3000]
 [  80  880]]
0.719029374201788
auc : 0.8083633273345331
recall : 0.9166666666666666
              precision    recall  f1-score   support

           0       0.99      0.70      0.82     10002
           1       0.23      0.92      0.36       960

    accuracy                           0.72     10962
   macro avg       0.61      0.81      0.59     10962
weighted avg       0.92      0.72      0.78     10962



In [10]:
model =ExtraTreesClassifier(n_estimators=200, criterion='gini', max_depth=40,
                            min_samples_split=5, min_samples_leaf=5, min_weight_fraction_leaf=0.0,  ## add main
                            max_features=20, max_leaf_nodes=None, min_impurity_decrease=0.0,
                            min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=-1,
                            random_state=None, verbose=0, warm_start=False, class_weight=None, 
                            ccp_alpha=0.0, max_samples=None)

model.fit(X_train, y_train)

y_predict = model.predict(X_val)
print(confusion_matrix(y_val,y_predict)) 
print(accuracy_score(y_val,y_predict))
    
fpr, tpr, thresholds = roc_curve(y_val, y_predict, pos_label=None)
print('auc :',auc(fpr, tpr))
print('recall :', recall_score(y_val, y_predict))
print(classification_report(y_val, y_predict))

[[7161 2841]
 [  97  863]]
0.7319832147418355
auc : 0.8074575709858028
recall : 0.8989583333333333
              precision    recall  f1-score   support

           0       0.99      0.72      0.83     10002
           1       0.23      0.90      0.37       960

    accuracy                           0.73     10962
   macro avg       0.61      0.81      0.60     10962
weighted avg       0.92      0.73      0.79     10962



In [11]:
model =XGBClassifier(base_score=0.3, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0.5,                           ## add main
              learning_rate=.1, max_delta_step=0, max_depth=5,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=61,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.7, verbosity=0)

model.fit(X_train, y_train)

y_predict = model.predict(X_val)
print(confusion_matrix(y_val,y_predict)) 
print(accuracy_score(y_val,y_predict))
    
fpr, tpr, thresholds = roc_curve(y_val, y_predict, pos_label=None)
print('auc :',auc(fpr, tpr))
print('recall :', recall_score(y_val, y_predict))
print(classification_report(y_val, y_predict))

[[7112 2890]
 [  74  886]]
0.7296113847837986
auc : 0.816987227554489
recall : 0.9229166666666667
              precision    recall  f1-score   support

           0       0.99      0.71      0.83     10002
           1       0.23      0.92      0.37       960

    accuracy                           0.73     10962
   macro avg       0.61      0.82      0.60     10962
weighted avg       0.92      0.73      0.79     10962



In [12]:
X_test = eda(data_test)

In [13]:
test_predict = model.predict(X_test)

In [14]:
data_test_copy['is_promoted'] = test_predict

In [15]:
data_test_copy[['employee_id','is_promoted']].to_csv('rgd_xgb.csv')